<a id='top'></a>

# CS312 S22


# Assignment 2 Coding

***

**Name**: 

***







**NOTES**: 

- You may not use late days on the exam coding nor can you drop your exam coding grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a exam coding clarifications thread. (NB: you should use public posts for the epidemiology "warmup" prompts).
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. The exam coding assignmnet is designed to be completed using only the packages in the first given code cell.
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.
- There is *not a prescribed API* for these problems.  You may answer coding questions with whatever syntax or object typing you deem fit.  Your evaluation will primarily live in the clarity of how well you present your final results, so don't skip over any interpretations!  Your code should still be commented and readable to ensure you followed the given course algorithm.
- There are two ways to quickly make a .pdf out of this notebook for Gradescope submission.  Either:
 - Use File -> Download as PDF via LaTeX.  This will require your system path find a working install of a TeX compiler
 - Easier: Use File ->  Print Preview, and then Right-Click -> Print using your default browser and "Print to PDF"

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Problem 3](#p3) | [Bottom](#bot)

---

In [2]:
#old stalwarts
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as stats
%matplotlib inline

#new friends
import seaborn as sns
import statsmodels.api as sm #this line is now always in our opener
from statsmodels.stats.outliers_influence import summary_table
from patsy import dmatrices # for making design matrices out of categorical features #instead of sm.add_constant


[Back to top](#top)

<a id='p1'></a>
## (40 pts) Problem 1: Simulation and Hypothesis Testing
***

One measure we're often interested in for modeling is whether or not our data "looks" normally distributed. We have explored two diagnoses for this briefly whereby we compare histograms of residuals or "QQ" plots of residuals to the corresponding results of the normal. 

In tihs problem we'll explore a simplified variant of another common diagnostic that's **more general** for whether or a not a list of numbers comes from a specified distribution.  I recommend skimming [this link](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) about the Kolmogorov-Smirnov test.  Before we begin.



### A) How Similar are two distributions?

Consider trying to tell somebody that the uniform random variable $X \sim U[-2,2]$ is "very different" from the Normal $Z \sim N(0,1)$.  How would you do so?  We could plot their *pdf*s, but the **height** of a pdf is pretty meaningless, since probability lives in **area**.  Instead, we could maybe compare their **cdfs**.

Your task:

- Create a `linspace` of 1,000 values from -4 to 4.
- At those values as the $x$-axis, make a plot with two lines: the cdf of $Z$ and the cdf of $X$ as given above
- Print $\arg \max_y |F_Z(y)-F_X(y)|$, the **maximum** value of the absolute *difference* between the cdfs $F_Z$ and $F_X$ at your linspace locations.  In other words: how far vertically apart were the two cdfs at their *most* far apart location?

In [3]:
space = np.linspace(-4, 4, num=1000)
y = stats.norm.cdf(space)



### B) How about *data* versus a distribution?

You should agree that it should maybe be possible to tell two random variables apart from their cdfs.  But what about data **from** a distribution?  This means we have noise or randomness, and it's harder to tell the difference between similar distributions!

Our tool for this is called the **empirical** cumulative density function ([ecdf]([https://en.wikipedia.org/wiki/Empirical_distribution_function])), which is roughly equivalent to the cdf version of a histogram: draw a picture of the *observed* cdf, which naturally will be a little "blocky" like a histogram.

Formally, the ecdf of a data set $X$ with $n$ observations is given by:

$$ecdf_X(x)=\frac{\text{# of elements of X that are less than or equal to x}}{n}$$

You can implement this function by hand, but you may also use `statsmodels.distributions.empirical_distribution.ECDF`.


Your task:

- **Simulate** 100 numbers from $X \sim U[-2,2]$ (see `stats.uniform.rvs` for syntax)
- Create a `linspace` of 1,000 values from -4 to 4.
- Using the `linspace` values as the $x$-axis, again make a plot with two lines: the cdf of the normal $Z$ but this time compared to the ecdf of the simulated $X$ values.
- Print $\arg \max_y |F_Z(y)-ecdf_X(y)|$, the **maximum** value of the absolute *difference* between the cdfs $F_Z$ and $F_X$ at your linspace locations.
- How does it compare to the "true" value in part A?  Run your code a few times and report at least 5 such values.

### C) How about for Normals?

The **distribution** of the numbers you compute in part $B$ is the basis of one measure for whether or not *data* fits a *proposed* distribution.  Our goal is to create a hypothesis test from scratch (recall HW6 # 3 for a similar concept) that gives us the ability to figure out how big of a distance we should get for these measures if the data **does** match the proposed distribution.

In other words, we're going to simulate from the null hypothesis and use it to apply to data.  Our motivating example is the `hubble.csv` data from HW7 and nb15, which had 24 observations.

**C1)** Your task:
- 1) Simulate 24 data points $X$ from the $N(4,10)$ distribution.
- 2) **Standardize** $X$ by converting it to a vector of Z-score: $X=\frac{X-\bar{X}}{S_X}$
- 3) At the same linspace as in parts A/B, compute the maximum distance between the ecdf of the standarized $X$ and the cdf of the N(0,1)$.
- Repeat steps 1-3 1000 times until you have a simulated distribution of max distances observed.


**C2) ** Make a histogram of your simulated max values observed.

**C3) ** Create a one-tailed hypothesis test corresponding to the *alternative* hypothesis of "the data does not come from the normal distribution" by using a test that rejects the null hypothesis if the distance between ecdf and cdf is $>c$ for some $c$.  If your tolerance for type I error is .10, what do you choose for $c$ on a data set of 24 observations?


**C4) ** Apply your hypothesis test in part C3 to the `hubble.csv` data by standardizing it and computing its ecdf's distance-from-normal.  The hubble data doesn't **look** very normal, but could we actually conclude that it definitely wasn't normal?  What does this tell you about data sets with only 24 observations?



---
[Back to top](#top)
<a id='p3'></a>
## (60 pts) Problems 2-3: Regression


The [Anage](https://genomics.senescence.info/species/index.html) database is a large repositiory of biological information, and includes various properties of various animals with an eye towards understanding the effects of aging in different species.  Most of the columns should be self-explanatory, but for more information you can consult their overview, [here](https://genomics.senescence.info/help.html#anage).

A couple of [acronyms](https://genomics.senescence.info/software/demographic.html) in the columns: IMR stands for "initial mortality rate," and represents non-age related deaths.  MRDT is the "mortality doubling rate," and is a measure for the age-related species deaths.

Unlike the *descriptive* analysis we did of this data set in the first exam coding, on this problem we will attempt to handle *prediction*.  As before, we will target the birth weight column, this time as the **response** to a linear model. 

The data is loaded in with a couple of preliminary explorations below.

In [3]:
df=pd.read_csv('../data/AnAge.csv', encoding='UTF-8')
print(df.columns) #information available: many numeric columns are blank for many species
print(df.shape) #4224 species available
df.head(3)

Index(['HAGRID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus',
       'Species', 'Common name', 'Female maturity (days)',
       'Male maturity (days)', 'Gestation/Incubation (days)', 'Weaning (days)',
       'Litter/Clutch size', 'Litters/Clutches per year',
       'Inter-litter/Interbirth interval', 'Birth weight (g)',
       'Weaning weight (g)', 'Adult weight (g)', 'Growth rate (1/days)',
       'Maximum longevity (yrs)', 'Source', 'Specimen origin', 'Sample size',
       'Data quality', 'IMR (per yr)', 'MRDT (yrs)', 'Metabolic rate (W)',
       'Body mass (g)', 'Temperature (K)', 'References'],
      dtype='object')
(4224, 31)


,HAGRID,Kingdom,Phylum,Class,Order,Family,Genus,Species,Common name,Female maturity (days),...,Source,Specimen origin,Sample size,Data quality,IMR (per yr),MRDT (yrs),Metabolic rate (W),Body mass (g),Temperature (K),References
0,3.0,Animalia,Arthropoda,Branchiopoda,Diplostraca,Daphniidae,Daphnia,pulicaria,Daphnia,NaN,...,NaN,unknown,medium,acceptable,NaN,NaN,NaN,NaN,NaN,1.294130e+11
1,5.0,Animalia,Arthropoda,Insecta,Diptera,Drosophilidae,Drosophila,melanogaster,Fruit fly,7.0,...,NaN,captivity,large,acceptable,0.05,0.04,NaN,NaN,NaN,2.203250e+37
2,6.0,Animalia,Arthropoda,Insecta,Hymenoptera,Apidae,Apis,mellifera,Honey bee,NaN,...,812,unknown,medium,acceptable,NaN,NaN,NaN,NaN,NaN,6.340740e+54


## (40 pts) Problem 2: Single-Predictor Regression

### A) A naive model
Suppose our goal is to predict the `Birth weight (g)` of a species using only its `Temperature (K)`.  

Your task:
- Create a new data frame with only these two columns.  Then drop any rows with missing observations.
-  Make a linear model with birth weight as the response, predicted by temperature.  Print the summary table, and write the final model in a markdown cell.
- Make a series of 3 side-by-side plots.  Lefthand plot: scatter plot of the data set and overlay the line of best least-squares fit.  Middle plot: histogram of residuals of best least-squares fit are the y-axis.  Righthand plot: scatter plot where the original x-values are the x-axis and the residuals are the y-axis.
- Does this model seem appropriate?  For each of the 4 major assumptions of the simple linear regression model, use your plots in part B to decide whether or not the data set meets those assumptions (don't spend too much time here, just look things over!)


### B) A better model

You should have **emphatically** decided that there were problems in your model.  But you may already have known this: the `Birth weight (g)` is logarithmic, so it might make sense for our erorrs to be as well.  One solution is to **replace** `Birth weight (g)` as our target with the logarithm of birth weight instead, and then try to predict *that* using `Temperature (K)`.

Your task: repeat all 5 steps of the analysis in part A, this time using the natural logarithm of `Birth weight (g)` as the response variable.
- Create a new data frame with only these two columns.  Then drop any rows with missing observations.
-  Make a linear model with log birth weight as the response, predicted by temperature.  Print the summary table, and write the final model in a markdown cell.
- Make a series of 3 side-by-side plots.  Lefthand plot: scatter plot of the data set and overlay the line of best least-squares fit.  Middle plot: histogram of residuals of best least-squares fit are the y-axis.  Righthand plot: scatter plot where the original x-values are the x-axis and the residuals are the y-axis.
- Does this model seem appropriate?  For each of the 4 major assumptions of the simple linear regression model, use your plots in part B to decide whether or not the data set meets those assumptions.
- Does your data set appear to have outliers?


#### Part C) Improvements: 

If the data set violated any assumptions in part B, adjust the model accordingly if possible.  Consider removing outliers, adding higher order polynomial terms, etc.  If you adjust the model, make another table and set of plots as in parts A and B of the new fit and the new residuals.


Then, write a sentence or two describing your thought process.  If you did not adjust a model, why not?  If you did, what are the effects?

<br>

---
[Back to top](#top)
<a id='p3'></a>

## [20 points] Problem 3: Go Big: Multiple Linear Regression

This problem is an open-ended problem where we will approach the same problem as in #2 but you may use more columns in the context of Multiple Linear Regression.  

Your goal is to demonstrate that you have an understanding of how to choose between columns and validate/check for problems in a multiple linear regression problem.  

The data set has been scrubbed to only include non-missing values of the following columns:
- The numerical column `Gestation/Incubation (days)`
- The numerical column `Litter/Clutch size`
- The numerical column `Maximum longevity (yrs)`
- The numerical column `Temperature (K)`
- The numerical column `Weaning (days)`

It also includes the **categorical** column `Order`, which you may use as a categorical predictor (use `pd.get_dummies`).  Use of this column make the problem harder, but will generate better fits.  Using it and carefully determining **which** orders can be used to help prediction is worth up to 5 points of extra credit.

The target/response is still `Birth weight (g)`, but as before you should apply a logarithm to it.

In [3]:
dfmlr=df[['Birth weight (g)','Common name' , 'Order', 'Gestation/Incubation (days)', 'Litter/Clutch size',\
          'Weaning (days)', 'Maximum longevity (yrs)', 'Metabolic rate (W)', 'Temperature (K)']]
dfmlr=dfmlr.dropna()
print(dfmlr.shape)
dfmlr.head(3)

(234, 9)


,Birth weight (g),Common name,Order,Gestation/Incubation (days),Litter/Clutch size,Weaning (days),Maximum longevity (yrs),Metabolic rate (W),Temperature (K)
1530,7.67,Lesser hedgehog tenrec,Afrosoricida,55.0,6.00,29.0,19.0,0.750,307.85
1532,11.50,Streaked tenrec,Afrosoricida,59.0,4.00,21.0,2.7,0.380,308.15
1533,3.95,Dobson's shrew tenrec,Afrosoricida,61.0,2.67,29.0,5.6,0.315,304.05




**Rubric Notes:**
This problem is by design very open-ended.  It is meant to reflect a real-world problem solving process.  For this problem, at the very least, you should:
- Include and explain which method you're using to remove redundant columns in part B
- Include a (short) sentence for **each** of the diagnositic plots in part C, determining whether your current model is meeting the standard assumptions of multiple linear regression
- Include some code, sentences, and/or visualizations demonstrating what *alternative models* you tried and/or considered.
- Include a sentence interpreting why your final model **makes sense** in the context of the problem - and it if doesn't, you'll want to try to explain why not.

Graders will look for the both the diagnostic plots and **plenty** of complete English sentences describing what you're doing and why you think it will help answer the data science question: how do we create the best model we can to quantify how price can be explained and predicted by the features of the house.

As a quick heuristic: you should be able to find models with adjusted $R^2$ to above 75% using only minor adjustments on the initially provided numerical columns, and above 90% if youuse the categorical column.

** Part 3 A: Explore**

Make pairwise scatter plots of the continuous predictors/covariates, both against each other and against the outcome (log weight).   Does the relationship between the independent variables and the dependent variables appear to be linear?  Do there appear to be independent variables that are collinear?

** Part 3 B: Make a Model**

By adding columns to a minimal model or by subtracting columns from the full model, use one of the criteria in the class to create a reasonable candidate model.  These may include:

- stepwise optimization of adjusted $R^2$
- stepwise inclusion/removal of most or least-significant T-tests on coefficients
- removing columns based on VIFs

Use a markdown cell to explain exactly what method you're using to construct your model.

** Part 3 C: Validate your Model**


Perform a thorough discussion of the underlying regression assumptions of your model in part 1B.  You should plot a predictor vs. residuals plot for each column and histogram OR qqplot of the overall residuals.  Make sure to also check for non-linearity, which lives in the predictor vs. residuals plots for each column.  

*Hint:* Consider applying similar functions to your predictor columns as you did to the response, as they're also in similar all-positive, right-skewed units.

** Part 3 D: Tune your Model**


Based on your work in parts 1B and 1C, **iterate** on your model.  Consider removing terms or adding higher-order polynomials one at a time unless you are satisfied that your model captures the data as well as possible.  Each time you add or subtract a term from your model, you should repeat the steps in parts B and C: a summary table and exploration of assumptions.

** Part 3 E: Explain your Model**

**Justify** your choices: there are a lot of ways to choose a "best" model: we've mentioned e.g. only including significant predictors versus F-tests versus optimizing R-squared.  Explain what terms you chose and why they were appropriate! 

<br>

---

[Back to top](#top)
<a id='bot'></a>